# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [85]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    n = nhl_df
    n['team'] = n['team'].str.strip()
    n['team'] = n['team'].replace("\*","",regex=True)
    n["team"] = n['team'].str.strip()
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n = n[n['W'].str.match(r'^[\d]')].reset_index(drop=True)
    change = {'Jackets':'Blue Jackets','Wings':'Red Wings','Leafs':'Maple Leafs','Knights':'Golden Knights'}
    n['area'] = n['team'].str.extract('(\w*$)').replace(change)
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    
    c = cities
    c['NHL'] = c['NHL'].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)
    c = c[c['NHL'].str.match(r'^[A-Z]')].rename(columns = {'Population (2016 est.)[8]':'Population'}).reset_index(drop=True)
    new = c.merge(n,how='outer',left_on='NHL',right_on='area').sort_values('team').reset_index(drop=True)
    new['Metropolitan area'].loc[0] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[13] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[18] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[19] = new['Metropolitan area'].loc[32]
    new['Population'].loc[0] = new['Population'].loc[31]
    new['Population'].loc[13] = new['Population'].loc[31]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[18] = new['Population'].loc[32]
    new['Population'].loc[19] = new['Population'].loc[32]
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    result.rename(columns={'ratio':'W/L ratio'})
    pop = new.iloc[:,[0,1]].set_index('Metropolitan area')
    tab = result.merge(pop,on='Metropolitan area')
    tab = tab[~tab.index.duplicated(keep='first')]
    tab['Population'] = pd.to_numeric(tab['Population'])
    # raise NotImplementedError()
    
    population_by_region = tab['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = tab['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr


(0.012486162921209907, 0.9497182859911791)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [86]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    big4 = 'NBA'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)
    
    n = nba_df
    n = n[n['year']==2018]
    n['team'] = n['team'].str.strip().replace("\*?\s\(\d*\)","",regex=True)
    n = n[['team','W','L']]
    n['t_id'] = n['team'].str.extract('(\w*$)')
    
    new = n.merge(c,how = 'outer',left_on='t_id',right_on='NBA')
    new = new[['Metropolitan area','Population','team','W','L','t_id','NBA']]
    
    new['Metropolitan area'].loc[10] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[11] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[24] = new['Metropolitan area'].loc[30]
    new['Metropolitan area'].loc[25] = new['Metropolitan area'].loc[30]
    new['Population'].loc[10] = new['Population'].loc[31]
    new['Population'].loc[11] = new['Population'].loc[31]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[24] = new['Population'].loc[30]
    new['Population'].loc[25] = new['Population'].loc[30]
    final = new
    final = final.loc[:29]
    final = final.sort_values('Metropolitan area').reset_index(drop=True)
    out=[]
    for group, frame in new.groupby('Metropolitan area'):
        twin = np.sum(pd.to_numeric(frame['W']))
        tloss = np.sum(pd.to_numeric(frame['L']))
        total = twin+tloss
        ratio = (twin/total)
        dict = {'Area' : group,'W/L ratio' : ratio}
        out.append(dict)

    ratio_df = pd.DataFrame(out)
    ratio_df = ratio_df.set_index('Area')
    final = final.set_index('Metropolitan area')
    final = final[~final.index.duplicated(keep='first')] #removes duplicated rows
    see = pd.concat([ratio_df,final],axis=1)
    see = see.iloc[:,[0,1]]
    see['Population'] = pd.to_numeric(see['Population'])
    
    # raise NotImplementedError()
    
    population_by_region = see['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = see['W/L ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr


-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    big4 = 'MLB'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)

    n = mlb_df
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n['t_id'] = n['team'].str.extract('(\w*$)')
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    
    new = n.merge(c,how = 'outer',left_on='t_id',right_on=big4)
    new = new.sort_values('team').reset_index(drop=True)
    new['Metropolitan area'].loc[3] = new['Metropolitan area'].loc[30]
    new['Metropolitan area'].loc[4] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[5] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[12] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[13] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[18] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[19] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[23] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[28] = new['Metropolitan area'].loc[35]
    new['Population'].loc[3] = new['Population'].loc[30]
    new['Population'].loc[4] = new['Population'].loc[31]
    new['Population'].loc[5] = new['Population'].loc[31]
    new['Population'].loc[12] = new['Population'].loc[32]
    new['Population'].loc[13] = new['Population'].loc[32]
    new['Population'].loc[17] = new['Population'].loc[33]
    new['Population'].loc[18] = new['Population'].loc[33]
    new['Population'].loc[19] = new['Population'].loc[34]
    new['Population'].loc[23] = new['Population'].loc[34]
    new['Population'].loc[28] = new['Population'].loc[35]
    
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    result.rename(columns={'ratio':'W/L ratio'})
    pop = new.iloc[:,[-2,-3]].set_index('Metropolitan area')
    tab = result.merge(pop,on='Metropolitan area')
    tab = tab[~tab.index.duplicated(keep='first')]
    tab['Population'] = pd.to_numeric(tab['Population'])
    #raise NotImplementedError()
     
    population_by_region = tab['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = tab['ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

(0.15027698302669307, 0.46370703378875583)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    big4 = 'NFL'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)

    n = nfl_df
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n['team'] = n['team'].str.strip().replace("\*?\+?","",regex=True)
    n['t_id'] = n['team'].str.extract('(\w*$)')
    n = n[n['W'].str.match(r'^[\d]')].reset_index(drop=True)
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    new = n.merge(c,how = 'outer',left_on='t_id',right_on=big4)
    #new = new[['team','Metropolitan area','Population','W','L','t_id',big4]]
    new = new.sort_values('team').reset_index(drop=True)

    new['Metropolitan area'].loc[16] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[22] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[23] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[24] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[27] = new['Metropolitan area'].loc[34]
    new['Population'].loc[16] = new['Population'].loc[32]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[22] = new['Population'].loc[33]
    new['Population'].loc[23] = new['Population'].loc[33]
    new['Population'].loc[24] = new['Population'].loc[34]
    new['Population'].loc[27] = new['Population'].loc[34]
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    result.rename(columns={'ratio':'W/L ratio'})
    pop = new.iloc[:,[-2,-3]].set_index('Metropolitan area')
    tab = result.merge(pop,on='Metropolitan area')
    tab = tab[~tab.index.duplicated(keep='first')]
    tab['Population'] = pd.to_numeric(tab['Population'])
    #  raise NotImplementedError()
    
    population_by_region = tab['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = tab['ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [83]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def get_nhl():
    n = nhl_df
    n['team'] = n['team'].str.strip()
    n['team'] = n['team'].replace("\*","",regex=True)
    n["team"] = n['team'].str.strip()
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n = n[n['W'].str.match(r'^[\d]')].reset_index(drop=True)
    change = {'Jackets':'Blue Jackets','Wings':'Red Wings','Leafs':'Maple Leafs','Knights':'Golden Knights'}
    n['area'] = n['team'].str.extract('(\w*$)').replace(change)
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    
    c = cities
    c['NHL'] = c['NHL'].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)
    c = c[c['NHL'].str.match(r'^[A-Z]')].rename(columns = {'Population (2016 est.)[8]':'Population'}).reset_index(drop=True)
    new = c.merge(n,how='outer',left_on='NHL',right_on='area').sort_values('team').reset_index(drop=True)
    new['Metropolitan area'].loc[0] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[13] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[18] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[19] = new['Metropolitan area'].loc[32]
    new['Population'].loc[0] = new['Population'].loc[31]
    new['Population'].loc[13] = new['Population'].loc[31]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[18] = new['Population'].loc[32]
    new['Population'].loc[19] = new['Population'].loc[32]
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    #result = result.rename(columns={'ratio':' NHL W/L ratio'})
    pop = new.iloc[:,[0,1]].set_index('Metropolitan area')
    tab = result.merge(pop,on='Metropolitan area')
    tab = tab[~tab.index.duplicated(keep='first')]
    tab['Population'] = pd.to_numeric(tab['Population'])
    tab = tab.reset_index()
    return tab

def get_nba():
    big4 = 'NBA'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)
        
    n = nba_df
    n = n[n['year']==2018]
    n['team'] = n['team'].str.strip().replace("\*?\s\(\d*\)","",regex=True)
    n = n[['team','W','L']]
    n['t_id'] = n['team'].str.extract('(\w*$)')
    
    new = n.merge(c,how = 'outer',left_on='t_id',right_on='NBA')
    new = new[['Metropolitan area','Population','team','W','L','t_id','NBA']]
    
    new['Metropolitan area'].loc[10] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[11] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[24] = new['Metropolitan area'].loc[30]
    new['Metropolitan area'].loc[25] = new['Metropolitan area'].loc[30]
    new['Population'].loc[10] = new['Population'].loc[31]
    new['Population'].loc[11] = new['Population'].loc[31]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[24] = new['Population'].loc[30]
    new['Population'].loc[25] = new['Population'].loc[30]
    final = new
    final = final.loc[:29]
    final = final.sort_values('Metropolitan area').reset_index(drop=True)
    out=[]
    for group, frame in new.groupby('Metropolitan area'):
        twin = np.sum(pd.to_numeric(frame['W']))
        tloss = np.sum(pd.to_numeric(frame['L']))
        total = twin+tloss
        ratio = (twin/total)
        dict = {'Area' : group,'ratio' : ratio}
        out.append(dict)

    ratio_df = pd.DataFrame(out)
    ratio_df = ratio_df.set_index('Area')
    final = final.set_index('Metropolitan area')
    final = final[~final.index.duplicated(keep='first')] #removes duplicated rows
    see = pd.concat([ratio_df,final],axis=1)
    see = see.iloc[:,[0,1]]
    see['Population'] = pd.to_numeric(see['Population'])
    see = see.reset_index()
    see = see.rename(columns = {'index':'Metropolitan area'})
    return see

def get_mlb(): 
    big4 = 'MLB'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)

    n = mlb_df
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n['t_id'] = n['team'].str.extract('(\w*$)')
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    
    new = n.merge(c,how = 'outer',left_on='t_id',right_on=big4)
    new = new.sort_values('team').reset_index(drop=True)
    new['Metropolitan area'].loc[3] = new['Metropolitan area'].loc[30]
    new['Metropolitan area'].loc[4] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[5] = new['Metropolitan area'].loc[31]
    new['Metropolitan area'].loc[12] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[13] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[18] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[19] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[23] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[28] = new['Metropolitan area'].loc[35]
    new['Population'].loc[3] = new['Population'].loc[30]
    new['Population'].loc[4] = new['Population'].loc[31]
    new['Population'].loc[5] = new['Population'].loc[31]
    new['Population'].loc[12] = new['Population'].loc[32]
    new['Population'].loc[13] = new['Population'].loc[32]
    new['Population'].loc[17] = new['Population'].loc[33]
    new['Population'].loc[18] = new['Population'].loc[33]
    new['Population'].loc[19] = new['Population'].loc[34]
    new['Population'].loc[23] = new['Population'].loc[34]
    new['Population'].loc[28] = new['Population'].loc[35]
    
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    #result = result.rename(columns={'ratio':'MLB W/L ratio'})
    pop = new.iloc[:,[-2,-3]].set_index('Metropolitan area')
    tab2 = result.merge(pop,on='Metropolitan area')
    tab2 = tab2[~tab2.index.duplicated(keep='first')]
    tab2['Population'] = pd.to_numeric(tab2['Population'])
    tab2 = tab2.reset_index()
    return tab2

def get_nfl(): 
    big4 = 'NFL'
    c = cities
    c[big4] = c[big4].replace("\[[\w\s]*\]","",regex=True)
    c = c[['Metropolitan area','Population (2016 est.)[8]',big4]].rename(columns = {'Population (2016 est.)[8]':'Population'})
    c = c[c[big4].str.match(r'^[A-Z\d]')]
    c = c.sort_values('Metropolitan area').reset_index(drop=True)

    n = nfl_df
    n = n[n['year']==2018]
    n = n[['team','W','L']]
    n['team'] = n['team'].str.strip().replace("\*?\+?","",regex=True)
    n['t_id'] = n['team'].str.extract('(\w*$)')
    n = n[n['W'].str.match(r'^[\d]')].reset_index(drop=True)
    n['ratio'] = pd.to_numeric(n['W'])/(pd.to_numeric(n['W'])+pd.to_numeric(n['L']))
    new = n.merge(c,how = 'outer',left_on='t_id',right_on=big4)
    #new = new[['team','Metropolitan area','Population','W','L','t_id',big4]]
    new = new.sort_values('team').reset_index(drop=True)

    new['Metropolitan area'].loc[16] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[17] = new['Metropolitan area'].loc[32]
    new['Metropolitan area'].loc[22] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[23] = new['Metropolitan area'].loc[33]
    new['Metropolitan area'].loc[24] = new['Metropolitan area'].loc[34]
    new['Metropolitan area'].loc[27] = new['Metropolitan area'].loc[34]
    new['Population'].loc[16] = new['Population'].loc[32]
    new['Population'].loc[17] = new['Population'].loc[32]
    new['Population'].loc[22] = new['Population'].loc[33]
    new['Population'].loc[23] = new['Population'].loc[33]
    new['Population'].loc[24] = new['Population'].loc[34]
    new['Population'].loc[27] = new['Population'].loc[34]
    result = new.groupby('Metropolitan area').agg({'ratio':np.mean})
    #result = result.rename(columns={'ratio':'NFL W/L ratio'})
    pop = new.iloc[:,[-2,-3]].set_index('Metropolitan area')
    tab3 = result.merge(pop,on='Metropolitan area')
    tab3 = tab3[~tab3.index.duplicated(keep='first')]
    tab3['Population'] = pd.to_numeric(tab3['Population'])
    tab3 = tab3.reset_index()
    
    return tab3

def get_df(k):
    if k == 'NFL': return get_nfl()
    elif k == 'NBA': return get_nba()
    elif k == 'NHL': return get_nhl()
    elif k == 'MLB': return get_mlb()
    else: print('Error')

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for a in sports:
        for b in sports:
            if a != b :
                df = pd.merge(get_df(a),get_df(b),on = ['Metropolitan area'])
                p_values.loc[a,b] = stats.ttest_rel(df['ratio_x'],df['ratio_y'])[1]
            
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
